# 出馬表から追切指数を出力するコード

In [32]:
import pandas as pd
import numpy as np
import glob
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 追切CSVファイルのパスを選択
file_path = filedialog.askopenfilename(title="追切CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("追切CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 追切CSVファイルを開く
traning_df = pd.read_csv(file_path, low_memory=False,encoding='cp932')

traning_df = traning_df.copy()

# 予想コメントのカラムを「|」で分割
split_columns = traning_df['予想コメント'].str.split('|', expand=True)
traning_df['日付'] = split_columns[0]
traning_df['コース'] = split_columns[1]
traning_df['馬場状態'] = split_columns[2]
traning_df['騎乗者'] = split_columns[3]
traning_df['タイム'] = split_columns[4]
traning_df['周回位置'] = split_columns[5]
traning_df['脚色'] = split_columns[6]

replace_map = {
    '南Ｗ': '美Ｗ',
    '南Ｄ': '美ダ',
    '南ダ': '美ダ',
    '南芝': '美芝'
}
traning_df['コース'] = traning_df['コース'].replace(replace_map)

# 7Fから1Fのカラムを np.nan で初期化
distance_columns = ['7F', '6F', '5F', '4F', '3F', '2F', '1F']
for col in distance_columns:
    traning_df[col] = np.nan  # 初期化

# 補正タイム用のカラムを追加
for col in distance_columns:
    traning_df[f'補正_{col}'] = np.nan  # 初期化

# 空文字列を NaN に変換
traning_df.replace("", pd.NA, inplace=True)

# 'コース', '馬場状態', '騎乗者' 列に欠損値がある行を削除（追切データが存在しないレコードを削除）
traning_df = traning_df.dropna(subset=['コース', '馬場状態', '騎乗者'])

# 各行のタイム情報を適切カラムに配置する処理
for index, row in traning_df.iterrows():
    times = row['タイム']
    if pd.isna(times):
        continue

    # タイムリストを取得
    time_list = times.split('-')
    time_length = len(time_list)

    # タイムを後ろから順に適切なカラムに配置
    for i in range(time_length):
        # 数値に変換できるかチェック
        try:
            traning_df.at[index, distance_columns[-(time_length - i)]] = float(time_list[i])
        except ValueError:
            # 数値に変換できない場合は NaN を代入
            traning_df.at[index, distance_columns[-(time_length - i)]] = np.nan

# 周り位置を数値型に変換
traning_df['周回位置'] = pd.to_numeric(traning_df['周回位置'], errors='coerce')

# タイム補正処理
for index, row in traning_df.iterrows():
    if not pd.isna(row['周回位置']):
        # 補正値を計算
        correction_value = (9 - row['周回位置']) * 0.1

        # 最も左に存在するタイムを探す
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)

            # 全体時計に補正値を加算
            traning_df.at[index, f'補正_{leftmost_col}'] = round(row[leftmost_col] + correction_value, 1)

            # 残り区間タイムに均等割り振り
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        traning_df.at[index, f'補正_{col}'] = round(row[col] + equal_correction, 1)
        else:
            # 周回位置があるがタイムが存在しない場合（通常は発生しない想定）
            continue
    else:
        # 周り位置がない場合、元のタイムを補正タイムに転記
        for col in distance_columns:
            if not pd.isna(row[col]):
                traning_df.at[index, f'補正_{col}'] = row[col]

# 出力に必要なカラムを選択
format_df = traning_df[['レースID(新)', '場所', '芝ダ', '距離', '外回り','競走種別C', 'クラスC',
                '調教師','調教師コード','コース', '馬場状態', '騎乗者','補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F' ,
                '周回位置', '脚色']].copy()

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# 競争種別とクラスコードを書き換え
format_df['競走種別C'] = format_df['競走種別C'].apply(categorize_race_type)
format_df['クラスC'] = format_df['クラスC'].apply(categorize_class_code)

# タイム関連のカラム
time_columns = ['補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F']

# タイム列を数値型に変換（必要に応じてNaNを処理）
format_df[time_columns] = format_df[time_columns].apply(pd.to_numeric, errors='coerce')

# マージするためにデータフレームのカラムをリネーム
format_df = format_df.rename(columns={'芝ダ': '芝・ダ','外回り': '芝(内・外)','競走種別C': '競走種別','クラスC': 'クラスコード'})

# 基準タイムファイルを読み込む
standardtime_filepath1 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Standard_Traningdata_*.csv'), key=str)
standard_time_data1 = pd.read_csv(standardtime_filepath1, encoding='utf-8')

standardtime_filepath2 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_median_Traningdata_*.csv'), key=str)
standard_time_data2 = pd.read_csv(standardtime_filepath2, encoding='utf-8')

standardtime_filepath3 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_median_Traningdata_*.csv'), key=str)
standard_time_data3 = pd.read_csv(standardtime_filepath3, encoding='utf-8')

standardtime_filepath4 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_median_Traningdata_*.csv'), key=str)
standard_time_data4 = pd.read_csv(standardtime_filepath4, encoding='utf-8')

standardtime_filepath5 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top20_Traningdata_*.csv'), key=str)
standard_time_data5 = pd.read_csv(standardtime_filepath5, encoding='utf-8')

standardtime_filepath6 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top10_Traningdata_*.csv'), key=str)
standard_time_data6 = pd.read_csv(standardtime_filepath6, encoding='utf-8')

standardtime_filepath7 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_top20_Traningdata_*.csv'), key=str)
standard_time_data7 = pd.read_csv(standardtime_filepath7, encoding='utf-8')

standardtime_filepath8 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_top10_Traningdata_*.csv'), key=str)
standard_time_data8 = pd.read_csv(standardtime_filepath8, encoding='utf-8')

standardtime_filepath9 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_top20_Traningdata_*.csv'), key=str)
standard_time_data9 = pd.read_csv(standardtime_filepath9, encoding='utf-8')

standardtime_filepath10 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_top10_Traningdata_*.csv'), key=str)
standard_time_data10 = pd.read_csv(standardtime_filepath10, encoding='utf-8')

standardtime_filepath11 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top20_Traningdata_*.csv'), key=str)
standard_time_data11 = pd.read_csv(standardtime_filepath11, encoding='utf-8')

standardtime_filepath12 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top10_Traningdata_*.csv'), key=str)
standard_time_data12 = pd.read_csv(standardtime_filepath12, encoding='utf-8')

# マージ条件
merge_keys1 = ['コース']
merge_keys2 = ['コース', '馬場状態']
merge_keys3 = ['競走種別','クラスコード','コース','馬場状態']
merge_keys4 = ['調教師コード','コース','馬場状態']

# 出馬表データと基準タイムファイル1を結合
merged_data = pd.merge(
    format_df,
    standard_time_data1,
    on=merge_keys1,
    how='left'
)

# 偏差値を計算する関数
def calculate_deviation(value, mean, std_dev):
    if pd.isna(value) or pd.isna(mean) or pd.isna(std_dev) or std_dev == 0:
        return np.nan  # 計算できない場合はNaNを返す
    return 50 + 10 * (mean - value) / std_dev

# 偏差値を計算して新しいカラムを作成
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'
    mean_col = col
    std_dev_col = f'標準偏差_{col}'
    deviation_col = f'偏差値_{col}'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル2との結合
merged_data = pd.merge(
    merged_data,  # 1つ目の基準タイムとの結合結果
    standard_time_data2,
    on=merge_keys2,  # マージ条件
    how='left',
    suffixes=['','_file2']
)

# 2つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file2'  # 基準タイム2の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム2の標準偏差
    deviation_col = f'偏差値_{col}_file2'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル3との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data3,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file3')
)

# 3つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file3'  # 基準タイム3の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム3の標準偏差
    deviation_col = f'偏差値_{col}_file3'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル4との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data4,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file4')
)

# 4つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file4'  # 基準タイム4の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム4の標準偏差
    deviation_col = f'偏差値_{col}_file4'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 総合偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    deviation_col1 = f'偏差値_{col}'  # 1つ目の基準タイム偏差値
    deviation_col2 = f'偏差値_{col}_file2'  # 2つ目の基準タイム偏差値
    deviation_col3 = f'偏差値_{col}_file3'  # 2つ目の基準タイム偏差値
    deviation_col4 = f'偏差値_{col}_file4'  # 2つ目の基準タイム偏差値
    overall_deviation_col = f'総合偏差値_{col}'  # 総合偏差値のカラム名
    merged_data[overall_deviation_col] = merged_data[[deviation_col1, deviation_col2,deviation_col3,deviation_col4]].mean(axis=1)
    
# 加点を計算する関数
def calculate_bonus(value, mean, is_saka, col):
    """
    value: 補正タイム
    mean: 基準タイム
    is_saka: 坂路コースかどうか
    col: 対象のハロンカラム（例: '4F', '2F', '1F'）
    """
    if pd.isna(value) or pd.isna(mean):
        return 0  # 比較できない場合は加点なし

    # 坂路と周回コースで加点基準を分ける
    if is_saka:
        if col == '4F':
            return 0.5 if value < mean else 0
        elif col == '2F':
            return 1.5 if value < mean else 0
        elif col == '1F':
            return 1.0 if value < mean else 0
    else:
        if col == '4F':
            return 0.5 if value < mean else 0
        elif col == '2F':
            return 1.0 if value < mean else 0
        elif col == '1F':
            return 1.5 if value < mean else 0
    return 0

# 基準タイムファイル5との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data5,
    on=merge_keys1,
    how='left',
    suffixes=('', '_file5')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file5'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'{col}_file5'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file5'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル6との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data6,
    on=merge_keys1,
    how='left',
    suffixes=('', '_file6')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file6'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'{col}_file6'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file6'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル7との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data7,
    on=merge_keys2,
    how='left',
    suffixes=('', '_file7')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file7'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file7'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file7'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル8との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data8,
    on=merge_keys2,
    how='left',
    suffixes=('', '_file8')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file8'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file8'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file8'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル9との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data9,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file9')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file9'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file9'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file9'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル10との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data10,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file10')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file10'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file10'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file10'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル11との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data11,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file11')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file11'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file11'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file11'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル12との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data12,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file12')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file12'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file12'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file12'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 総合スコアのカラムを動的に抽出
score_columns1 = [col for col in merged_data.columns if col.startswith('総合偏差値_')]
score_columns2 = [col for col in merged_data.columns if col.startswith('加点_')]

# 加点スコアのカラムを合計して「総合加点スコア」を作成
merged_data['総合偏差値スコア'] = merged_data[score_columns1].mean(axis=1)
merged_data['総合加点スコア'] = merged_data[score_columns2].sum(axis=1)

# '騎乗者係数'を追加
def assign_rider_coefficient(rider):
    if rider == '助手':
        return 1
    elif rider == '見習':
        return 0.8
    else:
        return 0.9

merged_data['騎乗者係数'] = merged_data['騎乗者'].apply(assign_rider_coefficient)

# '脚色係数'を追加
def assign_leg_action_coefficient(action):
    if pd.isna(action):  # 欠損値を確認
        return 0.9  # デフォルト値（他の場合と同じ処理）
    elif action == '馬也':
        return 1.1
    elif 'Ｇ' in str(action):  # NaNを回避するためにstr変換
        return 1.0
    elif '強' in str(action):  # NaNを回避するためにstr変換
        return 1.0
    elif action == '一杯':
        return 0.8
    else:
        return 0.9  # 上記以外の場合

merged_data['脚色係数'] = merged_data['脚色'].apply(assign_leg_action_coefficient)

# 追切指数を計算する関数
def calculate_training_index(row):
    base_score = row['総合偏差値スコア'] + row['総合加点スコア']
    adjusted_score = base_score * row['騎乗者係数'] * row['脚色係数']
    return adjusted_score

# 追切指数の計算
merged_data['追切指数'] = merged_data.apply(calculate_training_index, axis=1)

# 非有限値を置き換える
merged_data['追切指数'] = merged_data['追切指数'].replace([np.inf, -np.inf], np.nan)  # 無限値をNaNに置換
merged_data['追切指数'] = merged_data['追切指数'].fillna(1)  # NaNを1に置換

# 整数化とクリップ処理
merged_data['追切指数'] = merged_data['追切指数'].round(0).clip(lower=1, upper=99).astype(int)

merged_data = merged_data[['レースID(新)','追切指数']]

output_path1 = f'D:/Keiba/Importdata/30_Traning_Score/Traning_Score_{datetime.now().strftime("%Y%m%d")}.csv'
merged_data.to_csv(output_path1, index=False, encoding='utf_8_sig')
print(f"結果を{output_path1}に保存しました。")


結果をD:/Keiba/Importdata/30_Traning_Score/Traning_Score_20241215.csvに保存しました。


# 成績データファイルから追切指数を出力するコード

In [33]:
import pandas as pd
import numpy as np
import glob
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 追切CSVファイルのパスを選択
file_path = filedialog.askopenfilename(title="追切CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("追切CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 追切CSVファイルを開く
traning_df = pd.read_csv(file_path, low_memory=False,encoding='cp932')

traning_df = traning_df.copy()

# 予想コメントのカラムを「|」で分割
split_columns = traning_df['予想コメント'].str.split('|', expand=True)
traning_df['日付'] = split_columns[0]
traning_df['コース'] = split_columns[1]
traning_df['馬場状態'] = split_columns[2]
traning_df['騎乗者'] = split_columns[3]
traning_df['タイム'] = split_columns[4]
traning_df['周回位置'] = split_columns[5]
traning_df['脚色'] = split_columns[6]

replace_map = {
    '南Ｗ': '美Ｗ',
    '南Ｄ': '美ダ',
    '南ダ': '美ダ',
    '南芝': '美芝'
}
traning_df['コース'] = traning_df['コース'].replace(replace_map)

# 7Fから1Fのカラムを np.nan で初期化
distance_columns = ['7F', '6F', '5F', '4F', '3F', '2F', '1F']
for col in distance_columns:
    traning_df[col] = np.nan  # 初期化

# 補正タイム用のカラムを追加
for col in distance_columns:
    traning_df[f'補正_{col}'] = np.nan  # 初期化

# 空文字列を NaN に変換
traning_df.replace("", pd.NA, inplace=True)

# 'コース', '馬場状態', '騎乗者' 列に欠損値がある行を削除（追切データが存在しないレコードを削除）
traning_df = traning_df.dropna(subset=['コース', '馬場状態', '騎乗者'])

# 各行のタイム情報を適切カラムに配置する処理
for index, row in traning_df.iterrows():
    times = row['タイム']
    if pd.isna(times):
        continue

    # タイムリストを取得
    time_list = times.split('-')
    time_length = len(time_list)

    # タイムを後ろから順に適切なカラムに配置
    for i in range(time_length):
        # 数値に変換できるかチェック
        try:
            traning_df.at[index, distance_columns[-(time_length - i)]] = float(time_list[i])
        except ValueError:
            # 数値に変換できない場合は NaN を代入
            traning_df.at[index, distance_columns[-(time_length - i)]] = np.nan

# 周り位置を数値型に変換
traning_df['周回位置'] = pd.to_numeric(traning_df['周回位置'], errors='coerce')

# タイム補正処理
for index, row in traning_df.iterrows():
    if not pd.isna(row['周回位置']):
        # 補正値を計算
        correction_value = (9 - row['周回位置']) * 0.1

        # 最も左に存在するタイムを探す
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)

            # 全体時計に補正値を加算
            traning_df.at[index, f'補正_{leftmost_col}'] = round(row[leftmost_col] + correction_value, 1)

            # 残り区間タイムに均等割り振り
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        traning_df.at[index, f'補正_{col}'] = round(row[col] + equal_correction, 1)
        else:
            # 周回位置があるがタイムが存在しない場合（通常は発生しない想定）
            continue
    else:
        # 周り位置がない場合、元のタイムを補正タイムに転記
        for col in distance_columns:
            if not pd.isna(row[col]):
                traning_df.at[index, f'補正_{col}'] = row[col]

# 出力に必要なカラムを選択
format_df = traning_df[['レースID(新)', '場所', '芝・ダ', '距離', '芝(内・外)','競走種別', 'クラスコード',
                '調教師','調教師コード','コース', '馬場状態', '騎乗者','補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F' ,
                '周回位置', '脚色']].copy()

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# 競争種別とクラスコードを書き換え
format_df['競走種別'] = format_df['競走種別'].apply(categorize_race_type)
format_df['クラスコード'] = format_df['クラスコード'].apply(categorize_class_code)

# タイム関連のカラム
time_columns = ['補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F']

# タイム列を数値型に変換（必要に応じてNaNを処理）
format_df[time_columns] = format_df[time_columns].apply(pd.to_numeric, errors='coerce')

# マージするためにデータフレームのカラムをリネーム
##format_df = format_df.rename(columns={'芝ダ': '芝・ダ','外回り': '芝(内・外)','競走種別C': '競走種別','クラスC': 'クラスコード'})

# 基準タイムファイルを読み込む
standardtime_filepath1 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Standard_Traningdata_*.csv'), key=str)
standard_time_data1 = pd.read_csv(standardtime_filepath1, encoding='utf-8')

standardtime_filepath2 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_median_Traningdata_*.csv'), key=str)
standard_time_data2 = pd.read_csv(standardtime_filepath2, encoding='utf-8')

standardtime_filepath3 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_median_Traningdata_*.csv'), key=str)
standard_time_data3 = pd.read_csv(standardtime_filepath3, encoding='utf-8')

standardtime_filepath4 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_median_Traningdata_*.csv'), key=str)
standard_time_data4 = pd.read_csv(standardtime_filepath4, encoding='utf-8')

standardtime_filepath5 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top20_Traningdata_*.csv'), key=str)
standard_time_data5 = pd.read_csv(standardtime_filepath5, encoding='utf-8')

standardtime_filepath6 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top10_Traningdata_*.csv'), key=str)
standard_time_data6 = pd.read_csv(standardtime_filepath6, encoding='utf-8')

standardtime_filepath7 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_top20_Traningdata_*.csv'), key=str)
standard_time_data7 = pd.read_csv(standardtime_filepath7, encoding='utf-8')

standardtime_filepath8 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/41_course_top10_Traningdata_*.csv'), key=str)
standard_time_data8 = pd.read_csv(standardtime_filepath8, encoding='utf-8')

standardtime_filepath9 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_top20_Traningdata_*.csv'), key=str)
standard_time_data9 = pd.read_csv(standardtime_filepath9, encoding='utf-8')

standardtime_filepath10 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/43_class_top10_Traningdata_*.csv'), key=str)
standard_time_data10 = pd.read_csv(standardtime_filepath10, encoding='utf-8')

standardtime_filepath11 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top20_Traningdata_*.csv'), key=str)
standard_time_data11 = pd.read_csv(standardtime_filepath11, encoding='utf-8')

standardtime_filepath12 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top10_Traningdata_*.csv'), key=str)
standard_time_data12 = pd.read_csv(standardtime_filepath12, encoding='utf-8')

# マージ条件
merge_keys1 = ['コース']
merge_keys2 = ['コース', '馬場状態']
merge_keys3 = ['競走種別','クラスコード','コース','馬場状態']
merge_keys4 = ['調教師コード','コース','馬場状態']

# 出馬表データと基準タイムファイル1を結合
merged_data = pd.merge(
    format_df,
    standard_time_data1,
    on=merge_keys1,
    how='left'
)

# 偏差値を計算する関数
def calculate_deviation(value, mean, std_dev):
    if pd.isna(value) or pd.isna(mean) or pd.isna(std_dev) or std_dev == 0:
        return np.nan  # 計算できない場合はNaNを返す
    return 50 + 10 * (mean - value) / std_dev

# 偏差値を計算して新しいカラムを作成
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'
    mean_col = col
    std_dev_col = f'標準偏差_{col}'
    deviation_col = f'偏差値_{col}'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル2との結合
merged_data = pd.merge(
    merged_data,  # 1つ目の基準タイムとの結合結果
    standard_time_data2,
    on=merge_keys2,  # マージ条件
    how='left',
    suffixes=['','_file2']
)

# 2つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file2'  # 基準タイム2の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム2の標準偏差
    deviation_col = f'偏差値_{col}_file2'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル3との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data3,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file3')
)

# 3つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file3'  # 基準タイム3の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム3の標準偏差
    deviation_col = f'偏差値_{col}_file3'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 基準タイムファイル4との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data4,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file4')
)

# 4つ目の基準タイムを基準に偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    time_col = f'補正_{col}'  # 元データの補正タイム
    mean_col = f'補正_{col}_file4'  # 基準タイム4の平均タイム
    std_dev_col = f'標準偏差_補正_{col}'  # 基準タイム4の標準偏差
    deviation_col = f'偏差値_{col}_file4'
    merged_data[deviation_col] = merged_data.apply(
        lambda row: calculate_deviation(row[time_col], row[mean_col], row[std_dev_col]), axis=1
    )

# 総合偏差値を計算
for col in ['5F', '4F', '3F', '2F', '1F']:
    deviation_col1 = f'偏差値_{col}'  # 1つ目の基準タイム偏差値
    deviation_col2 = f'偏差値_{col}_file2'  # 2つ目の基準タイム偏差値
    deviation_col3 = f'偏差値_{col}_file3'  # 2つ目の基準タイム偏差値
    deviation_col4 = f'偏差値_{col}_file4'  # 2つ目の基準タイム偏差値
    overall_deviation_col = f'総合偏差値_{col}'  # 総合偏差値のカラム名
    merged_data[overall_deviation_col] = merged_data[[deviation_col1, deviation_col2,deviation_col3,deviation_col4]].mean(axis=1)
    
# 加点を計算する関数
def calculate_bonus(value, mean, is_saka, col):
    """
    value: 補正タイム
    mean: 基準タイム
    is_saka: 坂路コースかどうか
    col: 対象のハロンカラム（例: '4F', '2F', '1F'）
    """
    if pd.isna(value) or pd.isna(mean):
        return 0  # 比較できない場合は加点なし

    # 坂路と周回コースで加点基準を分ける
    if is_saka:
        if col == '4F':
            return 0.5 if value < mean else 0
        elif col == '2F':
            return 1.5 if value < mean else 0
        elif col == '1F':
            return 1.0 if value < mean else 0
    else:
        if col == '4F':
            return 0.5 if value < mean else 0
        elif col == '2F':
            return 1.0 if value < mean else 0
        elif col == '1F':
            return 1.5 if value < mean else 0
    return 0

# 基準タイムファイル5との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data5,
    on=merge_keys1,
    how='left',
    suffixes=('', '_file5')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file5'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'{col}_file5'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file5'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル6との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data6,
    on=merge_keys1,
    how='left',
    suffixes=('', '_file6')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file6'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'{col}_file6'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file6'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル7との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data7,
    on=merge_keys2,
    how='left',
    suffixes=('', '_file7')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file7'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file7'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file7'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル8との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data8,
    on=merge_keys2,
    how='left',
    suffixes=('', '_file8')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file8'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file8'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file8'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル9との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data9,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file9')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file9'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file9'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file9'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル10との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data10,
    on=merge_keys3,
    how='left',
    suffixes=('', '_file10')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file10'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file10'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file10'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル11との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data11,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file11')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file11'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file11'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file11'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 基準タイムファイル12との結合
merged_data = pd.merge(
    merged_data,
    standard_time_data12,
    on=merge_keys4,
    how='left',
    suffixes=('', '_file12')
)

# 加点用のカラムを初期化
for col in ['4F', '2F', '1F']:
    merged_data[f'加点_{col}_file12'] = 0  # 初期値を0に設定

# '坂'を含むコースかどうかを判定し、加点を適用
for col in ['4F', '2F', '1F']:
    time_col = f'補正_{col}'  # 補正タイム
    mean_col = f'補正_{col}_file12'  # 上位20%基準タイム
    
    # 加点計算を適用
    merged_data[f'加点_{col}_file12'] = merged_data.apply(
        lambda row: calculate_bonus(row[time_col], row[mean_col], '坂' in row['コース'] if not pd.isna(row['コース']) else False, col),
        axis=1
    )

# 総合スコアのカラムを動的に抽出
score_columns1 = [col for col in merged_data.columns if col.startswith('総合偏差値_')]
score_columns2 = [col for col in merged_data.columns if col.startswith('加点_')]

# 加点スコアのカラムを合計して「総合加点スコア」を作成
merged_data['総合偏差値スコア'] = merged_data[score_columns1].mean(axis=1)
merged_data['総合加点スコア'] = merged_data[score_columns2].sum(axis=1)

# '騎乗者係数'を追加
def assign_rider_coefficient(rider):
    if rider == '助手':
        return 1
    elif rider == '見習':
        return 0.8
    else:
        return 0.9

merged_data['騎乗者係数'] = merged_data['騎乗者'].apply(assign_rider_coefficient)

# '脚色係数'を追加
def assign_leg_action_coefficient(action):
    if pd.isna(action):  # 欠損値を確認
        return 0.9  # デフォルト値（他の場合と同じ処理）
    elif action == '馬也':
        return 1.1
    elif 'Ｇ' in str(action):  # NaNを回避するためにstr変換
        return 1.0
    elif '強' in str(action):  # NaNを回避するためにstr変換
        return 1.0
    elif action == '一杯':
        return 0.8
    else:
        return 0.9  # 上記以外の場合

merged_data['脚色係数'] = merged_data['脚色'].apply(assign_leg_action_coefficient)

# 追切指数を計算する関数
def calculate_training_index(row):
    base_score = row['総合偏差値スコア'] + row['総合加点スコア']
    adjusted_score = base_score * row['騎乗者係数'] * row['脚色係数']
    return adjusted_score

# 追切指数の計算
merged_data['追切指数'] = merged_data.apply(calculate_training_index, axis=1)

# 非有限値を置き換える
merged_data['追切指数'] = merged_data['追切指数'].replace([np.inf, -np.inf], np.nan)  # 無限値をNaNに置換
merged_data['追切指数'] = merged_data['追切指数'].fillna(1)  # NaNを1に置換

# 整数化とクリップ処理
merged_data['追切指数'] = merged_data['追切指数'].round(0).clip(lower=1, upper=99).astype(int)

# インポートデータ用に整形
merged_data = merged_data[['レースID(新)','追切指数']]

output_path1 = f'D:/Keiba/Importdata/30_Traning_Score/Traning_Score_all_{datetime.now().strftime("%Y%m%d")}.csv'
merged_data.to_csv(output_path1, index=False, encoding='utf_8_sig')
print(f"結果を{output_path1}に保存しました。")


結果をD:/Keiba/Importdata/30_Traning_Score/Traning_Score_all_20241215.csvに保存しました。


# スピード指数を出力するコード

## 概要
スピード指数を算出するためのコードを記録します。このスピード指数は競馬のレースごとに馬の成績を評価するための指標で、ペース補正、斤量補正、馬場指数、クラス補正など、複数の要素を組み合わせて算出します。

### 処理の概要

1. **CSVファイルの読み込み**
    - `pandas`を使って成績データのCSVファイルを読み込みます。このデータはレースごとの馬の成績や走破タイムなどが含まれています。

2. **データの前処理**

    a. **出走取り消し・競争除外の馬を除外**
    - 成績データから「外」「止」「消」の馬を除外します。

    b. **走破タイムの秒数への変換**
    - 走破タイム（`分.秒.10分の1秒`形式）を秒数（浮動小数点数）に変換する関数`convert_time_to_seconds`を使い、`走破タイム_秒`という新しいカラムに変換後の値を追加します。

    c. **コーナーロスの計算**
    - 各馬のコーナーでのロス距離を考慮した走破タイムの補正を行います。`calculate_corner_loss`関数を用いて、`コーナーロス`という新しいカラムにロス時間を計算して追加します。
    - 調整後の走破タイムは`調整走破タイム`というカラムに記録されます。

3. **クラスと競争種別の分類**
    - 各レースにおける競争種別コードとクラスコードをもとに分類し、新たに`Race_Type_Category`と`Class_Code_Category`カラムを追加します。

4. **基準タイムデータとのマージ**
    - コースごとの基準タイムデータ（StandardTimes2）およびクラス別基準タイムデータ（StandardTimes3）を読み込み、対象の成績データとマージします。
    - マージする際に、カラム名の整合性を取るためにリネーム処理を行います。

5. **スローorハイ関数の計算**
    - 新しいカラム`スローorハイ関数`を計算し、`Ave-3F`と`上り3F`の差を計算します。その結果を基に`スローorハイ関数範囲`を設定します。
    - `スローorハイ関数範囲`はペース補正ファイルと統一するために、あらかじめ定義した範囲で値を設定します。

6. **ペース補正ファイルとのマージ**
    - ペース補正ファイル（PacecorrectionTimes1）を読み込み、ペース補正係数を対象の成績データに結合します。

7. **各補正値の計算**

    a. **タイム補正値の計算**
    - `Top3_Finish_Time_standard`と`調整走破タイム`を基に、距離補正を考慮してタイム補正値を計算し、`タイム補正値`カラムに保存します。

    b. **平均3Fタイム補正値の計算**
    - `Top3_Average_3F`がNaNの場合、0に置き換えた上で、`Ave-3F`との差分を計算し、`Ave-3F補正値`として保存します。

    c. **斤量補正値の計算**
    - 斤量と基準斤量を用いて斤量補正を計算し、`斤量補正値`として保存します。

    d. **クラス補正値の計算**
    - `Top3_Finish_Time_standard`と`Top3_Finish_Time_class`の差を計算してクラス補正値を求めます。クラス補正値は範囲を-2から+2に収めるため、`np.clip`を使用して値を制限します。

    e. **ペース補正値の計算**
    - `スローorハイ関数`と`ペース補正係数`を乗算して、`ペース補正値`として保存します。`ペース補正係数`がNaNの場合は0に置き換えます。

8. **スピード指数の計算**
    - 各補正値を組み合わせて、最終的なスピード指数を算出します。
    - 計算式は以下の通りです：
    
      ```
      スピード指数 = タイム補正値 + Ave-3F補正値 + 馬場指数 + 斤量補正値 + クラス補正値 + ペース補正値 + 70
      ```
    - 計算後、`スピード指数`カラムに保存します。

9. **結果の保存**
    - 最終的に計算された`スピード指数`を含むデータを新たなCSVファイルとして保存します。
    - ファイル名には日付が付与され、指定のディレクトリに保存されます。

### 出力
- 出力ファイルには以下のカラムが含まれます。
  - レースID(新)
  - スピード指数
- 保存先：`D:/Keiba/Importdata/40_Speed_Score/SpeedScore_YYYYMMDD.csv`



In [1]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime
import glob
import re

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# スピード指数用成績CSVファイルのパスを選択して読み込み
file_path = filedialog.askopenfilename(title="スピード指数用成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("スピード指数用成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

df = pd.read_csv(file_path, low_memory=False, encoding='cp932')

# 出走取り消しや競争除外（外、止、消）になった馬を除外
df = df[~df['着順'].isin(['外', '止', '消'])]

# 走破タイムを時間（秒）に変換する関数
def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    # ドットで分割する (例: "1.54.5" → ["1", "54", "5"])
    parts = time_str.split(".")
    if len(parts) == 3:
        try:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
            return total_seconds
        except ValueError:
            return np.nan
    else:
        return np.nan

# 走破タイムを秒に変換
df['走破タイム_秒'] = df['走破タイム'].apply(convert_time_to_seconds)

# コーナーロスを計算する関数
def calculate_corner_loss(row):
    corner_positions = [
        row.get('馬印4', 1) - 1,
        row.get('馬印5', 1) - 1,
        row.get('馬印6', 1) - 1,
        row.get('馬印7', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # 総距離ロス（m）
    finish_time_seconds = row['走破タイム_秒']
    distance_m = row['距離']

    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan

    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 馬印4～7を数値に変換し、不正な値を1に設定
for corner_col in ['馬印4', '馬印5', '馬印6', '馬印7']:
    df[corner_col] = pd.to_numeric(df[corner_col], errors='coerce').fillna(1).astype(int)

# コーナーロスを計算して新しいカラムに追加
df['コーナーロス'] = df.apply(calculate_corner_loss, axis=1)

# 調整走破タイムの計算
df['調整走破タイム'] = df['走破タイム_秒'] - df['コーナーロス']

# 馬場指数の計算
df['馬場指数'] = df['レース印１'] / 10

# 基準斤量の計算
# 斤量を数値に変換する関数
def extract_weight(weight_str):
    match = re.search(r'\d+', weight_str)
    return int(match.group()) if match else np.nan

# 斤量を数値に変換
df['斤量'] = df['斤量'].apply(extract_weight)

# 基準斤量の計算
df['基準斤量'] = df['斤量'] - df['馬齢斤量差']

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# Target出力ファイルに競争種別とクラスコードの分類カラムを追加
df['Race_Type_Category'] = df['競走種別'].apply(categorize_race_type)
df['Class_Code_Category'] = df['クラスコード'].apply(categorize_class_code)

# スローorハイ関数の範囲を定義
ranges = [
    (-np.inf, -4.6),
    (-4.5, -3.6),
    (-3.5, -2.6),
    (-2.5, -1.6),
    (-1.5, -0.6),
    (-0.5, 0.5),
    (0.6, 1.5),
    (1.6, 2.5),
    (2.6, 3.5),
    (3.6, 4.5),
    (4.6, np.inf)
]

# スローorハイ関数のラベルを定義 (ペース補正ファイルと統一する)
labels = [
    '-inf～-4.6', '-4.5～-3.6', '-3.5～-2.6', '-2.5～-1.6', '-1.5～-0.6',
    '-0.5～0.5', '0.6～1.5', '1.6～2.5', '2.6～3.5', '3.6～4.5', '4.6～inf'
]

# Track_Condition の変換
df['Track_Condition_Category'] = df['馬場状態'].apply(lambda x: '良・稍' if x in ['良', '稍'] else '重・不' if x in ['重', '不'] else np.nan)

# スローorハイ関数を計算
df['スローorハイ関数'] = df['Ave-3F'] - df['上り3F']

# スローorハイ関数のラベルを設定
df['スローorハイ関数範囲'] = pd.cut(df['スローorハイ関数'], bins=[r[0] for r in ranges] + [ranges[-1][1]], labels=labels, right=True)

# ベースとなるデータフレーム
index_df = df[['レースID(新)',
            '場所',
            '芝・ダ',
            '距離',
            'Race_Type_Category',
            'Class_Code_Category',
            'Track_Condition_Category',
            '馬場指数',
            '基準斤量',
            '斤量',
            '走破タイム_秒',
            '調整走破タイム',
            'Ave-3F',
            '上り3F',
            'スローorハイ関数',
            'スローorハイ関数範囲'
            ]]

# コース基準タイムファイルのパスを取得（StandardTimes2_を含む最新ファイルを取得）
standard_time_file_path1 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes3_*.csv'), key=str)

# クラス別コース基準タイムファイルのパスを取得（StandardTimes3_を含む最新ファイルを取得）
standard_time_file_path2 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes2_*.csv'), key=str)

# ペース補正ファイルのパスを取得（10_PacecorrectionTimes1_を含む最新ファイルを取得）
pace_setting_file_path = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/10_PacecorrectionTimes1_*.csv'), key=str)

# コース基準タイムファイルの読み込み
standard_times_df1 = pd.read_csv(standard_time_file_path1, low_memory=False)
standard_times_df2 = pd.read_csv(standard_time_file_path2, low_memory=False)

# ペース補正ファイルの読み込み
pace_setting_df = pd.read_csv(pace_setting_file_path, low_memory=False)

# マージのためのカラム名マッピング（基準タイムファイル/Target出力ファイル）
column_mappings = {
    'Location': '場所',
    'Turf_Dirt': '芝・ダ',
    'Distance': '距離',
    'Track_Condition_Category': '馬場状態',
    'Race_Type_Category':'競争種別',
    'Class_Code_Category' : 'クラスコード'
}


# 指数データフレームのカラムリネーム
index_df = index_df.rename(columns=column_mappings)

# 各基準タイムファイルとマージするためのカラムリネーム
standard_times_df1 = standard_times_df1.rename(columns=column_mappings)
standard_times_df2 = standard_times_df2.rename(columns=column_mappings)

# ペース補正データフレームのカラムリネーム
pace_setting_df = pace_setting_df.rename(columns=column_mappings)

# 基準タイムファイルをマージ
merged_df = pd.merge(
    index_df,
    standard_times_df1[['場所', '芝・ダ', '距離', '馬場状態','Top3_Average_3F', 'Top3_Finish_Time','Distance_Index']],
    on=['場所', '芝・ダ', '距離', '馬場状態'],
    how='left'
)

# クラス別基準タイムファイルをマージ
merged_df = pd.merge(
    merged_df,
    standard_times_df2[['場所', '芝・ダ', '距離', '馬場状態', '競争種別', 'クラスコード', 'Top3_Finish_Time']],
    on=['場所', '芝・ダ', '距離', '馬場状態', '競争種別', 'クラスコード'],
    how='left',
    suffixes=('_standard', '_class')
)

# ペース補正ファイルをマージ
merged_df = pd.merge(
    merged_df,
    pace_setting_df[['場所', '芝・ダ', '距離', 'スローorハイ関数範囲', 'ペース補正係数']],
    on=['場所', '芝・ダ', '距離', 'スローorハイ関数範囲'],
    how='left',
)

# タイム補正値の計算
merged_df['タイム補正値'] = (merged_df['Top3_Finish_Time_standard'] - merged_df['調整走破タイム']) * merged_df['Distance_Index']

# 平均3Fタイム補正値の計算
# 平均3FタイムがNaNの場合は0に置き換え
merged_df['Top3_Average_3F'] = merged_df['Top3_Average_3F'].fillna(0)

# 平均3Fタイム補正値の計算 (Top3_Average_3FがNaNの場合はAve-3F補正値を0にする)
merged_df['Ave-3F補正値'] = np.where(
    merged_df['Top3_Average_3F'] == 0,
    0,
    merged_df['Top3_Average_3F'] - merged_df['Ave-3F']
)

# 斤量補正値の計算
merged_df['斤量補正値'] = (merged_df['斤量'] - merged_df['基準斤量']) / merged_df['Distance_Index']

# クラス補正値の計算
merged_df['クラス補正値'] = merged_df['Top3_Finish_Time_standard'] - merged_df['Top3_Finish_Time_class']

# クラス補正値を -2 から +2 の範囲に収める
merged_df['クラス補正値'] = np.clip(merged_df['クラス補正値'], -2, 2)

# ペース補正値の計算
# ペース補正係数がNaNの場合は0に置き換え
merged_df['ペース補正係数'] = merged_df['ペース補正係数'].fillna(0)
merged_df['ペース補正値'] = merged_df['スローorハイ関数'] * merged_df['ペース補正係数']

# スピード指数の計算
merged_df['スピード指数'] = round(merged_df['タイム補正値'] + merged_df['Ave-3F補正値'] + merged_df['馬場指数'] + merged_df['斤量補正値'] + merged_df['クラス補正値'] + merged_df['ペース補正値'] + 70,1)

# スピード指数のデータフレーム
speedindex_df = merged_df[['レースID(新)','スピード指数']]

# データフレームをcsvファイルで保存
output_path = f'D:/Keiba/Importdata/40_Speed_Score/SpeedScore_{datetime.now().strftime("%Y%m%d")}.csv'
speedindex_df.to_csv(output_path, index=False, encoding='utf_8_sig')
print(f"結果を{output_path}に保存しました。")

結果をD:/Keiba/Importdata/40_Speed_Score/SpeedScore_20241201.csvに保存しました。


# レースレベルを計算するコード

In [15]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime
import glob

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 成績CSVファイルのパスを選択して読み込み
file_path = filedialog.askopenfilename(title="[merged_update]成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("[merged_update]成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# レースレベル基準ファイルのパスを取得（RaceLevels_を含む最新ファイルを取得）
racelevel_filepath = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/20_RaceLevels_*.csv'), key=str)

race_df = pd.read_csv(file_path, low_memory=False)
standard_df = pd.read_csv(racelevel_filepath, low_memory=False)

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# 成績ファイルに競争種別の分類カラムを追加
race_df['Race_Type_Category'] = race_df['Race_Type'].apply(categorize_race_type)
race_df['Class_Code_Category'] = race_df['Class_Code'].apply(categorize_class_code)

# 上位3頭の値を平均化する
race_df['Top3_Adjustment'] = race_df[['1st_Horse_Adjustment', '2nd_Horse_Adjustment', '3rd_Horse_Adjustment']].mean(axis=1)
race_df['Top3_Adjustment9'] = race_df[['1st_Horse_Additional9', '2nd_Horse_Additional9', '3rd_Horse_Additional9']].mean(axis=1)
race_df['Top3_Speed_Index'] = race_df[['1st_Horse_Speed_Index', '2nd_Horse_Speed_Index', '3rd_Horse_Speed_Index']].mean(axis=1)

# 基準ファイルの重み付けと一致させる
weights = {
    'Top3_Adjustment': 0.2,
    'Top3_Adjustment9': 0.4,
    'Top3_Speed_Index': 0.4
}

# 重み付けを適用して平均スコアを計算
race_df['Average_Score'] = (
    race_df['Top3_Adjustment'] * weights['Top3_Adjustment'] +
    race_df['Top3_Adjustment9'] * weights['Top3_Adjustment9'] +
    race_df['Top3_Speed_Index'] * weights['Top3_Speed_Index']
)

# レース条件でデータをマージ
merge_columns = ['Turf_Dirt', 'Distance', 'Race_Type_Category', 'Class_Code_Category']
merged_df = pd.merge(race_df, standard_df, on=merge_columns, how='left')

# 近似値照合ロジックを実装する関数
def assign_race_level(row):
    # Average_Scoreを取得
    avg_score = row['Average_Score']
    # A～Eの基準値を取得
    thresholds = {
        'A': row['A'],
        'B': row['B'],
        'C': row['C'],
        'D': row['D'],
        'E': row['E']
    }

    # 平均スコアと各基準値の差分を計算
    differences = {key: abs(avg_score - value) for key, value in thresholds.items()}

    # 最小の差分を持つランクを取得
    closest_rank = min(differences, key=differences.get)

    # 対応する表記を生成
    rank_to_label = {'A': '05A', 'B': '07B', 'C': '01C', 'D': '00D', 'E': '03E'}
    return rank_to_label[closest_rank]

# Race_Levelカラムを追加
merged_df['Race_Level'] = merged_df.apply(assign_race_level, axis=1)

# 出力ディレクトリとファイル名の設定
output_dir = r'D:/Keiba/Importdata/20_Race_Level'
output_file = f'{output_dir}/race_levels_{datetime.now().strftime("%Y%m%d")}.csv'

# 出力データフレームを選択
output_df = merged_df[['Target_Race_ID', 'Race_Level']]

# 出力処理
output_df.to_csv(output_file, index=False, encoding='cp932')
print(f"レースレベル判定結果を {output_file} に保存しました。")


レースレベル判定結果を D:/Keiba/Importdata/20_Race_Level/race_levels_20241207.csv に保存しました。


# 33ラップからペース判定を計算するコード

In [18]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime
import glob

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 成績CSVファイルを選択
file_path = filedialog.askopenfilename(title="[merged_update]成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("[merged_update]成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 33ラップ基準ファイルのパスを取得（Standard33Laps_を含む最新ファイルを取得）
racelevel_filepath = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/30_Standard33Laps_*.csv'), key=str)

race_df = pd.read_csv(file_path, low_memory=False)
standard_df = pd.read_csv(racelevel_filepath, low_memory=False)

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# Target出力ファイルに競争種別の分類カラムを追加
race_df['Race_Type_Category'] = race_df['Race_Type'].apply(categorize_race_type)

# 共通のカラムで結合
merge_columns = ['Location', 'Turf_Dirt', 'Distance','Turf_Inside_Outside', 'Race_Type_Category']
merged_df = pd.merge(race_df, standard_df, on=merge_columns, how='left')

# カラム名の調整
merged_df = merged_df.rename(columns={'33_Lap_x': '33_Lap', '33_Lap_y': '33_Lap-0'})

# 各基準値との差分を計算
merged_df['diff_33_Lap-2'] = abs(merged_df['33_Lap'] - merged_df['33_Lap-2'])
merged_df['diff_33_Lap-1'] = abs(merged_df['33_Lap'] - merged_df['33_Lap-1'])
merged_df['diff_33_Lap']   = abs(merged_df['33_Lap'] - merged_df['33_Lap-0'])
merged_df['diff_33_Lap+1'] = abs(merged_df['33_Lap'] - merged_df['33_Lap+1'])
merged_df['diff_33_Lap+2'] = abs(merged_df['33_Lap'] - merged_df['33_Lap+2'])

def assign_label(row):
    # 差分辞書
    diffs = {
        'diff_33_Lap-2': row['diff_33_Lap-2'],
        'diff_33_Lap-1': row['diff_33_Lap-1'],
        'diff_33_Lap':   row['diff_33_Lap'],
        'diff_33_Lap+1': row['diff_33_Lap+1'],
        'diff_33_Lap+2': row['diff_33_Lap+2']
    }

    # 最小差分キーを取得
    min_key = min(diffs, key=diffs.get)

    # キー→数値マッピング
    mapping = {
        'diff_33_Lap-2': -2,
        'diff_33_Lap-1': -1,
        'diff_33_Lap':   0,
        'diff_33_Lap+1': 1,
        'diff_33_Lap+2': 2
    }

    scale = mapping[min_key]
    lap_value = row['33_Lap']

    # 0スケールの場合
    if scale == 0:
        if lap_value < 0:
            return '持0'
        elif lap_value > 0:
            return '瞬0'
        else:
            return '総'

    # 0以外の場合
    prefix = '瞬' if lap_value > 0 else '持'
    if scale > 0:
        suffix = f'+{scale}'
    else:
        suffix = f'{scale}'

    return prefix + suffix

merged_df['33_Lap_Type'] = merged_df.apply(assign_label, axis=1)

# ラベル書き換え用のマッピング辞書
label_mapping = {
    '持-2': '0T持-2',
    '持-1': '0T持-1',
    '持0': '0U持0',
    '持+1': '0V持+1',
    '持+2': '0V持+2',
    '瞬-2': '0S瞬-2',
    '瞬-1': '0S瞬-1',
    '瞬0': '0R瞬0',
    '瞬+1': '0Q瞬+1',
    '瞬+2': '0Q瞬+2',
    '総': '02総'
}

# mapを用いてラベル書き換え
merged_df['33_Lap_Type'] = merged_df['33_Lap_Type'].map(label_mapping)

# 出力用のデータフレーム
output_df = merged_df[['Target_Race_ID', '33_Lap_Type']]

# CSVファイルで保存
output_path = f'D:/Keiba/Importdata/20_33Lap_Type/33Laps_Type_{datetime.now().strftime("%Y%m%d")}.csv'
output_df.to_csv(output_path, index=False, encoding='cp932')
print(f"結果を{output_path}に保存しました。")


結果をD:/Keiba/Importdata/20_33Lap_Type/33Laps_Type_20241207.csvに保存しました。
